In [59]:
from working_with_files import *
from preprocessing import *
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from collections import Counter

In [122]:
def sort_by_official_dates(df):
    df1 = pd.DataFrame() # < 2008
    df2 = pd.DataFrame() # 2008 - 2013
    df3 = pd.DataFrame() # 2013 - 2016
    df4 = pd.DataFrame() # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date <= date1:
                df1 = df1.append(row, ignore_index=True)
            if date > date1 and date <= date2:
                df2 = df2.append(row, ignore_index=True)
            if date > date2 and date <= date3:
                df3 = df3.append(row, ignore_index=True)
            if date > date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def sort_by_unofficial_dates(df):
    df1 = pd.DataFrame() # < 2004
    df2 = pd.DataFrame() # 2004 - 2008
    df3 = pd.DataFrame() # 2008 - 2014
    df4 = pd.DataFrame() # > 2014
    date1 = datetime.strptime('01-01-2004', '%d-%m-%Y').date()
    date2 = datetime.strptime('01-01-2008', '%d-%m-%Y').date()
    date3 = datetime.strptime('01-01-2014', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date < date1:
                df1 = df1.append(row, ignore_index=True)
            if date >= date1 and date < date2:
                df2 = df2.append(row, ignore_index=True)
            if date >= date2 and date < date3:
                df3 = df3.append(row, ignore_index=True)
            if date >= date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

def get_stat(dfs):
    n = 0
    for df in dfs:
        print('Corpus {}:'.format(n), len(df), 'texts', len(get_all_texts(df).split()), 'words')
        n += 1

def show_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic {}:'.format(topic_idx))
        print(' '.join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print()

def get_topics(df, n_topics, n_top_words, models, stop_topics):
    
    documents = df.loc[:, 'text_clean']
    
    random_seed = 42
    
    if 'nmf' in models:
        tfidf = tfidf_vectorizer.fit_transform(documents)
        tfidf_feature_names = tfidf_vectorizer.get_feature_names()
        
        nmf = NMF(n_components=n_topics, random_state=random_seed, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
        
        print('NMF')
        show_topics(nmf, tfidf_feature_names, n_top_words)
        
        doc_topic = nmf.transform(tfidf)
        
    if 'lda' in models:
        tf = tf_vectorizer.fit_transform(documents)
        tf_feature_names = tf_vectorizer.get_feature_names()
        
        lda = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online', learning_offset=50., 
                                random_state=random_seed).fit(tf)
        
        print('LDA')
        show_topics(lda, tf_feature_names, n_top_words)
        
        doc_topic = lda.transform(tf)
    
    topics = []    
    for n in range(doc_topic.shape[0]):
        # topic = doc_topic[n].argmax()
        top_topics = doc_topic[n].argsort()[-3:][::-1] # topics numbers
        if top_topics[0] in stop_topics:
            top_topic = top_topics[1]
        else:
            top_topic = top_topics[0]
        # print(doc_topic[n][:3]) # probabilities
        # topic = sorted(doc_topic[n], reverse=True)[0:2]
        # print("doc: {} topic: {}\n".format(n, topic))
        topics.append(top_topic)
    df['topic_number'] = topics
        
    return df

### Sorting data by official dates

In [48]:
# read df and split it into 4 separate df4 based on publication date
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs = sort_by_official_dates(df)

In [49]:
# statistics 
get_stat(sorted_dfs)

Corpus 0: 1583 texts 845988 words
Corpus 1: 3168 texts 1111520 words
Corpus 2: 3344 texts 1082001 words
Corpus 3: 4277 texts 1420393 words


In [50]:
# get unnecessary symbols to remove them from texts
symbols_to_del = generate_symbols_to_del(get_all_texts(df))
symbols_to_del.extend(list('abcdefghijklmnopqrstuvwxyz'))
print(symbols_to_del)

['́', ')', '(', '<', '»', '´', '―', '/', '#', '‑', '_', '°', '0', '–', '.', '>', '{', '－', '[', '1', '?', '2', ':', '*', '\n', '3', '6', '@', ']', '−', '€', '„', '’', '·', '&', '4', '8', ',', '7', '`', '+', '!', ';', '“', '«', '$', '=', '！', '—', '"', '”', '‒', '…', '%', '№', '9', '\xad', '5', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [51]:
# clean texts from unnecessary symbols

for df in sorted_dfs:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [52]:
# get and preprocess stopwords
# source: ruscorpora.ru

stopwords_raw = get_lexicon_from_file('russian_stopwords.txt')
stopwords = []
stopwords_string = ''.join(stopwords_raw)
strings = stopwords_string.split('\t')
for string in strings:
    if string.isalpha():
        stopwords.append(string)

# lemmatize stopwords

stopwords_lemmatized = [lemmatize_pymorphy(word) for word in stopwords]
stopwords_lemmatized = list(set(stopwords_lemmatized))
stopwords = [stopword for stopword in stopwords_lemmatized if stopword.isalpha()]

stopwords.extend('''б г д е ж з й л м н п р т ф х ц ч ш щ ъ ы ь э ю
                наш твой мой свой ваш его их ее над чей такой другой между год страна государство также все отношение г-н 
                россия российский сторона де два дж мид внешнеполитический интервью министр газета иностранный дело
                катерина спецкор новость рассказывать подобный кроме министерство кстати пресс-конференция москва сайт лавров
                медведев путин сайт подробный рф s форум сердечно добрый желать приветствовать сам ясный ясность кто-то
                говорить хотеть официальный еще какой-то об лишь включая вокруг ле никакой во'''.split())

print(stopwords)

['что', 'ничто', 'мочь', 'же', 'ну', 'надо', 'более', 'тут', 'нет', 'без', 'о', 'там', 'только', 'бы', 'где', 'за', 'один', 'то', 'кто', 'раз', 'ли', 'от', 'к', 'весь', 'очень', 'здесь', 'тот', 'а', 'есть', 'вы', 'этот', 'вот', 'она', 'при', 'ещё', 'тогда', 'уже', 'из', 'когда', 'это', 'потому', 'мы', 'с', 'до', 'в', 'у', 'по', 'на', 'чем', 'я', 'не', 'чтобы', 'себя', 'ты', 'даже', 'они', 'он', 'и', 'так', 'для', 'который', 'тем', 'время', 'ни', 'через', 'но', 'под', 'теперь', 'после', 'человек', 'или', 'если', 'как', 'быть', 'сказать', 'потом', 'можно', 'б', 'г', 'д', 'е', 'ж', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'наш', 'твой', 'мой', 'свой', 'ваш', 'его', 'их', 'ее', 'над', 'чей', 'такой', 'другой', 'между', 'год', 'страна', 'государство', 'также', 'все', 'отношение', 'г-н', 'россия', 'российский', 'сторона', 'де', 'два', 'дж', 'мид', 'внешнеполитический', 'интервью', 'министр', 'газета', 'иностранный', 'дело', 'катерина', 'с

In [53]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b['\w-]+\b")
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=10, stop_words=stopwords, token_pattern=r"\b['\w-]+\b")

In [123]:
df1 = get_topics(sorted_dfs[0], 33, 15, ['nmf'], [0, 2]) # до 2008

NMF
Topic 0:
международный развитие новый политика мир мировой интерес проблема становиться решение работа глобальный важный безопасность экономический
Topic 1:
израиль палестинец дорожный карта израильский палестино-израильский палестинский четверка посредник квартет урегулирование пна израильтянин ближневосточный аббас
Topic 2:
сотрудничество вопрос обсуждать переговоры визит президент договариваться двусторонний встреча договоренность взаимодействие сегодня сфера торговый-экономический коллега
Topic 3:
палестинский пна аббас хамас администрация территория межпалестинский глава правительство национальный народ израиль обстановка мирный палестинец
Topic 4:
европа совет се европейский председательство конвенция комитет парламентский правовой общеевропейский право ассамблея разделительный организация демократия
Topic 5:
исламский цивилизация оик ислам религия религиозный православный мусульманский мир церковь культура мусульманин межцивилизационный конференция диалог
Topic 6:
соотечеств

In [124]:
df2 = get_topics(sorted_dfs[1], 39, 15, ['nmf'], [0, 2, 11]) # 2008 - 2013

NMF
Topic 0:
международный мир развитие глобальный новый мировой безопасность становиться работа интерес политика основа решение система проблема
Topic 1:
осетия южный абхазия республика закавказье женевский граница грузия абхазский неприменение дискуссия югоосетинский сила соглашение цхинвал
Topic 2:
сотрудничество двусторонний взаимодействие обсуждать переговоры вопрос сфера область коллега торговый-экономический связь сегодня совместный развитие договоренность
Topic 3:
русский язык центр культура фонд россотрудничество школа мир фестиваль русскоязычный рубеж китай образование китайский университет
Topic 4:
соотечественник рубеж конференция зарубежный конгресс проживать всемирный правительственный община диаспора координационный комиссия русский работа федерация
Topic 5:
сша американский российский-американский вашингтон про обама клинтон госдепартамент вопрос госсекретарь администрация президент штат снв государственный
Topic 6:
снг содружество заседание глава совет государство-учас

In [125]:
df3 = get_topics(sorted_dfs[2], 38, 15, ['nmf'], [0, 3]) # 2013 - 2016

NMF
Topic 0:
международный развитие сотрудничество регион работа интерес экономический новый сегодня усилие мир безопасность проблема основа важный
Topic 1:
сирийский сирия оппозиция боевик террорист алеппо сара дамаск вооруженный группировка правительство джабхат женевский группа район
Topic 2:
украина украинский киев власть киевский юго-восток минский ситуация заявление донецкий донбасс действие договоренность февраль донецк
Topic 3:
визит двусторонний сотрудничество федерация вопрос встреча переговоры ход обсуждать актуальный состояться взаимодействие повестка республика рабочий
Topic 4:
погибший выздоровление соболезнование пострадавший осуждать решительно террористический погибать выражать близкие скорый теракт родные ранение результат
Topic 5:
сша американский вашингтон администрация санкция против госдепартамент российский-американский керри про обама заявление белый санкционный госсекретарь
Topic 6:
оон резолюция сб совет безопасность генеральный право международный сессия прин

In [126]:
df4 = get_topics(sorted_dfs[3], 42, 15, ['nmf'], [4, 5, 0]) # С 2016

NMF
Topic 0:
друг сотрудничество сегодня уважаемый президент коллега рад важный международный регион интерес взаимодействие развитие контакт диалог
Topic 1:
сша американский вашингтон договор санкция американец военный вооружение администрация против госдепартамент заявление нато ракета дрсмд
Topic 2:
сирийский сирия турция астанинский процесс урегулирование оон резолюция иран сб конгресс оппозиция сочи астана встреча
Topic 3:
украина украинский киев киевский донбасс власть минский закон режим действие миссия порошенко нынешний право наблюдательный
Topic 4:
визит двусторонний сотрудничество переговоры федерация республика актуальный обсуждать региональный вопрос рабочий повестка ход мнение состояться
Topic 5:
мероприятие молодой участие дипломат федерация представитель принимать дипломатический состояться приглашать международный федеральный конференция посвящать зарубежный
Topic 6:
сирия сирийский террорист боевик провинция гуманитарный район зона житель лагерь пункт военный помощь ми

In [127]:
topics_labels = get_json_from_file('topics_labels.json')
topics_labels_upd = {}
for indx, label_to_topics in topics_labels.items():
    topic_to_label = {}
    for label, topics in label_to_topics.items():
        for topic in topics:
            topic_to_label[topic] = label
    topics_labels_upd[indx] = topic_to_label
    
dfs = [df1, df2, df3, df4]

n = 0
for df in dfs:
    print('Dataframe', n)
    nums_rate = dict(sorted(dict(Counter(list(df.loc[:, 'topic_number']))).items(), key=lambda kv: kv[1], reverse=True))
    labels_rate = {}
    topic_to_label = topics_labels_upd[n]
    for topic, freq in nums_rate.items():
        label = topic_to_label[topic]
        if label not in labels_rate:
            labels_rate[label] = freq
        else:
            labels_rate[label] += freq
    print(sorted(labels_rate.items(), key=lambda kv: kv[1], reverse=True))
    n += 1

Dataframe 0
[('Ближний Восток', 217), ('Постсов', 189), ('Европа', 171), ('Экономика', 149), ('Общие темы', 142), ('ООН', 93), ('Ядерная безопасность', 84), ('Терроризм', 73), ('США', 71), ('ВОВ', 71), ('Исламский мир', 44), ('Африка', 41), ('НАТО', 41), ('Арктика', 40), ('Русская культура', 36), ('Право', 35), ('Соотечественники за рубежом', 33), ('Азия', 29), ('ЮНЕСКО', 24)]
Dataframe 1
[('Ближний Восток', 648), ('Соотечественники за рубежом', 513), ('Постсов', 440), ('Общие темы', 259), ('Азия', 251), ('Европа', 242), ('Ядерная безопасность', 217), ('ВОВ', 149), ('США', 113), ('Право', 91), ('Африка', 83), ('Гуманитарная помощь', 71), ('Арктика', 55), ('Таиланд', 36)]
Dataframe 2
[('Постсов', 706), ('Ближний Восток', 507), ('Соотечественники за рубежом', 250), ('Общие темы', 243), ('ВОВ', 191), ('Европа', 191), ('Терроризм', 183), ('США', 163), ('Ядерная безопасность', 134), ('Африка', 118), ('Латинская Америка', 107), ('ООН', 106), ('Гуманитарная помощь', 81), ('НАТО', 79), ('Боинг

In [81]:
for indx, row in df1.iterrows():
    if row['topic_number'] in [0, 2]:
        print(row['text'], '\n')

Само подписание соглашения заранее анонсировалось и неожиданностью для нас не является. Нынешняя администрация США под предлогом мнимой иранской ракетной угрозы упорно ведет линию на развертывание в Европе третьего позиционного района (ТПР) своей глобальной ПРО. Примечательно, что все это происходит на фоне серьезных противоречий в чешском обществе и, конечно, не добавляет безопасности ни Чехии, ни Европе в целом.
Позиция России на этот счет хорошо известна – мы выступаем против американского проекта ТПР, способного подорвать стабильность и безопасность не только в общеевропейском, но и в глобальном масштабе.
Вызывает разочарование, что наша альтернатива – создание подлинно коллективной системы безопасности от ракетных угроз – по сути проигнорирована. Все наши аргументы были лишь выслушаны, но отнюдь не приняты во внимание.
Если договоренности с США, подлежащие ратификации в чешском парламенте, все-таки приобретут юридически обязывающий характер и у наших границ начнется реальное разве

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 

Министр иностранных дел Армении В.Осканян уже изложил все темы нашего сегодняшнего разговора, вопросы, которые затрагивались на встрече с Президентом Армении Р.Кочаряном и содержание беседы с Премьер-министром Армении А.Маркаряном. Действительно, год у нас ожидается весьма насыщенным для партнерских, союзнических отношений. Предстоит выполнить договоренности, которые в декабре прошлого года выработала Межправительственная комиссия по экономическому сотрудничеству. Это - расширение торговли, экономическое сотрудничество, включая инвестиционные проекты. Есть договоренности по военно-техническому сотрудничеству. Условились ускорить проработку всех вопросов, о которых договорились стороны, с тем, чтобы наши президенты, когда в ближайшее время будет открыт Год России в Армении, смогли бы посмотреть, как выполняются их принципиальные решения. Также предстоит встреча двух президентов в Москве на праздновании 60-летия Победы над фашизмом. Запланированы контакты на уровне министров иностранн

### Sorting data by unofficial dates

In [256]:
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')
sorted_dfs2 = sort_by_unofficial_dates(df)
get_stat(sorted_dfs2)

Corpus 0: 199 texts 78201 words
Corpus 1: 1228 texts 691187 words
Corpus 2: 3849 texts 1372091 words
Corpus 3: 7096 texts 2318423 words


In [257]:
for df in sorted_dfs2:
    df['text_clean'] = df['text_lemmatized'].apply(clean, args=(symbols_to_del,))

In [258]:
get_topics(sorted_dfs2[0].loc[:, 'text_clean'], 20, 15, ['nmf']) # До 2004

NMF
Topic 0:
сотрудничество развитие международный работа проблема вопрос организация новый проект мировой важный взаимодействие сфера деятельность регион
Topic 1:
палестинский дорожный карта израиль пна четверка израильский посредник ближневосточный палестинец палестино-израильский урегулирование правительство возобновление мир
Topic 2:
ирак иракский санкция ситуация гуманитарный поездка ущерб посольство заместитель программа война гражданин работать вопрос экономический
Topic 3:
комиссия социальный член гражданин организация заседание программа история федерация научный возможность господин служба утверждение семья
Topic 4:
договор стратегический ратификация сша государственный федеральный федерация российский-американский собрание стабильность президент комитет потенциал близкий фактор
Topic 5:
террористический осуждать соболезнование акт решительно акция погибший жертва выражать взрыв сочувствие сообщение результат иметься терроризм
Topic 6:
транзит калининградский документ соглаше

In [259]:
get_topics(sorted_dfs2[1].loc[:, 'text_clean'], 35, 15, ['nmf']) # 2004 - 2008

NMF
Topic 0:
развитие сотрудничество международный экономический работа новый важный решение организация задача регион являться интерес становиться проблема
Topic 1:
оон резолюция безопасность совет сб право международный устав генеральный член организация правозащитный урегулирование секретарь сессия
Topic 2:
сотрудничество вопрос обсуждать визит переговоры двусторонний президент договариваться договоренность торговый-экономический встреча ход сфера сегодня взаимодействие
Topic 3:
обсе организация бдипча смид выборы безопасность государство-участник реформа корзина мониторинг наблюдение наблюдатель бюро председательство обязательство
Topic 4:
политика мир внешний международный мировой война холодный глобальный современный интерес дипломатия коллективный новый угроза цивилизация
Topic 5:
палестинский израиль палестинец пна израильский аббас квартет дорожный карта территория палестино-израильский хамас урегулирование посредник ближневосточный
Topic 6:
соотечественник русский язык рубеж 

In [260]:
get_topics(sorted_dfs2[2].loc[:, 'text_clean'], 38, 15, ['nmf']) # 2008 - 2014

NMF
Topic 0:
международный развитие сотрудничество новый работа мир интерес сегодня глобальный проблема важный основа мировой задача решение
Topic 1:
сирия сирийский оппозиция сара вооруженный дамаск женевский аннан насилие правительство прекращение лаг сириец боевик химический
Topic 2:
визит сотрудничество переговоры двусторонний федерация обсуждать республика вопрос состояться президент ход взаимодействие встреча рабочий актуальный
Topic 3:
осетия грузия южный абхазия грузинский тбилиси республика саакашвили закавказье август граница сила абхазский югоосетинский неприменение
Topic 4:
соотечественник рубеж конференция всемирный зарубежный конгресс проживать правительственный координационный федерация община диаспора комиссия работа русский
Topic 5:
оон резолюция сб совет безопасность генеральный международный решение организация ассамблея миссия сессия генассамблея генсекретарь секретарь
Topic 6:
гражданин сша американский посольство ребенок консульский суд власть информация сотрудник

In [261]:
get_topics(sorted_dfs2[3].loc[:, 'text_clean'], 65, 15, ['nmf']) # С 2014

NMF
Topic 0:
международный интерес мир проблема безопасность регион усилие решение работа развитие основа процесс принцип диалог новый
Topic 1:
сша американский вашингтон санкция администрация американец госдепартамент против санкционный российский-американский давление обама госсекретарь вводить военный
Topic 2:
сирия сирийский оппозиция урегулирование процесс сириец межсирийский женева правительство террорист сара политический переговоры астанинский группа
Topic 3:
украина украинский киев киевский власть юго-восток действие заявление нынешний язык верховный одесса очередной порошенко кризис
Topic 4:
визит актуальный переговоры двусторонний повестка рабочий вопрос обсуждать мнение региональный международный республика федерация обмен день
Topic 5:
война победа великий память мировой второй отечественный история летие нацизм ветеран исторический народ нацистский мероприятие
Topic 6:
химический озхо оружие применение запрещение химоружие вещество кзхо сирия конвенция отравлять муфс докл